In [194]:
# Cleaning image names and meta data 
import re 
import os 
import shutil

img_dir = '/Volumes/SDD/hipsci_images'
save_dir = '/Volumes/SDD/hipsci_images/ALL_IMAGES'

for folder in os.listdir(img_dir):
    
    if '.DS_Store' not in folder: 
        print('Donor: ', folder)
        donor = folder
        
        for sub_folder in os.listdir(str(img_dir + '/' + folder)):
            
            if '.DS_Store' not in sub_folder: 
  
                print('Donor', folder, '|| Concentration:', sub_folder)
                conc = sub_folder
                
                temp_e = ''
                time_t = 1
                
                for image in os.listdir(str(img_dir + '/' + folder + '/' + sub_folder)):
                    
                    imagename = image.split('.')[0]
                    print(imagename)
                    timestamp = imagename.split('y')[1]
                    
                    m = timestamp.split('m')[0] # month
                    d = timestamp.split('m')[1].split('d')[0] # day
                    h = timestamp.split('_')[1].split('h')[0] # hour 
                    t = timestamp.split('h')[1].split('m')[0] # time
                    
                    st = str(m + d +  h +  t)
                    r = "456results string789"
                    s = ''.join(x for x in st if x.isdigit())
                    time_s = int(s[1:])
                    
                    exp = imagename.split('_')[1].split('_1_2')[0]
                    
                    if exp == temp_e and time_s > time_t:
                        count = count + 1
                    else:
                        count = 1
                        
                    print "HIPSCI CELL| ",  donor, "|conc: ", conc, "|exp: ", exp, "|time: ", time_s, "|hour: ", count
                    
                    temp_e = exp
                    time_t = time_s 
                    
                    dst_path = (save_dir + "/HIPSCI-d-" + donor + "-c-" + conc + "-e-" + exp + "-t-" + str(time_s) + "-hr-" + str(count) + ".tif")
                    src_path = (img_dir + "/" + folder + "/" + sub_folder + "/" +image)
#                     print(src_path, dst_path)
                    shutil.copy(src_path, dst_path)

('Donor: ', 'ALL_IMAGES')
('Donor: ', 'Bob')
('Donor', 'Bob', '|| Concentration:', 'FN1')
2014-05-20 HipSci Exp 5_A1_1_2014y05m20d_19h00m
HIPSCI CELL|  Bob |conc:  FN1 |exp:  A1 |time:  5201900 |hour:  1
2014-05-20 HipSci Exp 5_A1_1_2014y05m20d_20h00m
HIPSCI CELL|  Bob |conc:  FN1 |exp:  A1 |time:  5202000 |hour:  2
2014-05-20 HipSci Exp 5_A1_1_2014y05m20d_21h00m
HIPSCI CELL|  Bob |conc:  FN1 |exp:  A1 |time:  5202100 |hour:  3
2014-05-20 HipSci Exp 5_A1_1_2014y05m20d_22h00m
HIPSCI CELL|  Bob |conc:  FN1 |exp:  A1 |time:  5202200 |hour:  4
2014-05-20 HipSci Exp 5_A1_1_2014y05m20d_23h00m
HIPSCI CELL|  Bob |conc:  FN1 |exp:  A1 |time:  5202300 |hour:  5
2014-05-20 HipSci Exp 5_A1_1_2014y05m21d_00h00m
HIPSCI CELL|  Bob |conc:  FN1 |exp:  A1 |time:  5210000 |hour:  6
2014-05-20 HipSci Exp 5_A1_1_2014y05m21d_01h00m
HIPSCI CELL|  Bob |conc:  FN1 |exp:  A1 |time:  5210100 |hour:  7
2014-05-20 HipSci Exp 5_A1_1_2014y05m21d_02h00m
HIPSCI CELL|  Bob |conc:  FN1 |exp:  A1 |time:  5210200 |hour:  

In [41]:
import tflearn

In [32]:
def label_img(img):
    label = img.split('.')
    if "L0" in label[0]: 
        return [0,1]
    if "L1" in label[0]:
        return [1, 0]

In [33]:
def create_train_data():
    training_data = []

    for img in os.listdir(TRAIN_DIR):
        
        label = label_img(img)
        path = os.path.join(TRAIN_DIR, img)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
#         img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img), np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    
    return training_data



In [34]:
def process_test_data():
    testing_data = []
    
    for img in os.listdir(TEST_DIR):
        path = os.path.join(TEST_DIR, img)
        img_num = img.split('.')[0]
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
#         img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])

    shuffle(testing_data)
    np.save('test_dat.npy', testing_data)
    return testing_data 

In [35]:
train_data = create_train_data()

In [36]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

# Output = 2, either or normal or abnormal cell
convnet = fully_connected(convnet, 2, activation='softmax')

# set learning rate = LR 
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')



In [37]:
#load preloaded model
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded.')

In [39]:
# train data is all data minus last 500
train = train_data[0:500]
# test data is last 500 data
test = train_data[-500:]

X = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
test_y = [i[1] for i in test]

model.fit({'input': X}, {'targets': Y}, n_epoch=3, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=500, show_metric=True, run_id=MODEL_NAME)

ValueError: cannot reshape array of size 500 into shape (50,50,1)

In [24]:
print train[0][0]

[[135 134 134 ..., 126 126 126]
 [134 134 134 ..., 126 126 126]
 [133 134 134 ..., 126 126 126]
 ..., 
 [128 129 129 ..., 128 128 128]
 [129 129 129 ..., 128 128 128]
 [129 129 129 ..., 128 128 128]]
